In [1]:
import json

data_path = './lmsys.json'  # TODO
with open(data_path) as f:
    data = json.load(f)


In [2]:
new_file_cues = './parsed_data_cues.jsonl'
new_file_roles = './parsed_data_roles.jsonl'

# token to signal the end of the assistant's response
separator = '</s>'

all_prompts_cues = []
all_prompts_roles = []
for conv in data:
    complete_prompt = ''
    for i, turn in enumerate(conv['conversations']):
        complete_prompt = complete_prompt.replace('all cuee',
                                                  'all cues')  # FIXME fix this in script that creates lmsys.json
        if turn['from'] == 'human':
            complete_prompt += 'User: '
            complete_prompt += turn['value']
        elif turn['from'] == 'gpt':
            complete_prompt += 'Assistant: '
            sample = json.dumps({'input': complete_prompt, 'output': turn['value'] + separator})
            if i == 1 and sample not in all_prompts_cues:  # TODO simplify or document this
                all_prompts_cues.append(sample)
            elif i == 3 and sample not in all_prompts_cues:
                all_prompts_roles.append(sample)
            elif i != 1 and i != 3:
                print('something wrong here')
            complete_prompt += turn['value'] + separator
        complete_prompt += '\n'

# write parsed prompts to file
with open(new_file_cues, 'w') as f:
    f.write('\n'.join(all_prompts_cues))

with open(new_file_roles, 'w') as f:
    f.write('\n'.join(all_prompts_roles))

In [3]:
# check that the file with the cue prompts was written correctly
with open(new_file_cues) as f:
    lines = f.readlines()

print(f'Number of samples: {len(lines)}\n')

print('First 5 samples:')
for l in lines[:5]:
    print('in:', json.loads(l)['input'])
    print('out:', json.loads(l)['output'])
    print()

9399
in: User: A cue is the lexical items in a sentence that indicate that speech, writing, or thought is being reproduced.
I want you to extract all cues in the text below.
If you find multiple words for one cue, you output them separated by commas.
If no cue can be found in the given text, you output the string #UNK# as cue.
Now extract all cues from the following sentence.
Use the prefix "Cues: ".
Sentence: Frau Präsidentin !
Assistant: 
out: Cues: #UNK#</s>

in: User: A cue is the lexical items in a sentence that indicate that speech, writing, or thought is being reproduced.
I want you to extract all cues in the text below.
If you find multiple words for one cue, you output them separated by commas.
If no cue can be found in the given text, you output the string #UNK# as cue.
Now extract all cues from the following sentence.
Use the prefix "Cues: ".
Sentence: Liebe Kolleginnen und Kollegen !
Assistant: 
out: Cues: #UNK#</s>

in: User: A cue is the lexical items in a sentence that i

In [4]:
# check that the file with the role prompts was written correctly
with open(new_file_roles) as f:
    lines = f.readlines()

print(f'Number of samples: {len(lines)}\n')

print('First 5 samples:')
for l in lines[:5]:
    print('in:', json.loads(l)['input'])
    print('out:', json.loads(l)['output'])
    print()

5914
in: User: A cue is the lexical items in a sentence that indicate that speech, writing, or thought is being reproduced.
I want you to extract all cues in the text below.
If you find multiple words for one cue, you output them separated by commas.
If no cue can be found in the given text, you output the string #UNK# as cue.
Now extract all cues from the following sentence.
Use the prefix "Cues: ".
Sentence: Bundeskanzlerin Angela Merkel hat auf der Klimakonferenz in Bonn gesprochen .
Assistant: Cues: [gesprochen]</s>
User: Now I give you again the sentence only in addition with the two following sentences, because the roles can be partially contained in the following sentences.
Text: Bundeskanzlerin Angela Merkel hat auf der Klimakonferenz in Bonn gesprochen . Sie hat dort den Klimawandel als eine zentrale Herausforderung für die Menschheit bezeichnet . Sie hat von einer Schicksalsfrage gesprochen .

Now find all roles in the sentence associated with the cue 'gesprochen' you found i

In [5]:
# encode all prompt inputs with the Llama 1 tokenizer (same as the Llama 2 tokenizer)
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained(
    'huggyllama/llama-7b',
    padding_side="right",
    use_fast=False,  # Fast tokenizer giving issues.
    tokenizer_type='llama'
)

lines_cues = []
lines_roles = []
with open(new_file_cues) as f:
    lines_cues.extend(f.readlines())
with open(new_file_roles) as f:
    lines_roles.extend(f.readlines())

encodings_cues = []
encodings_roles = []
for l in lines_cues:
    enc = tokenizer.encode(json.loads(l)['input'])
    encodings_cues.append(enc)
for l in lines_roles:
    enc = tokenizer.encode(json.loads(l)['input'])
    encodings_roles.append(enc)

You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [6]:
# check the length of the encoded prompts, needed to set the max length of the model in the fine-tuning scripts
max_length_cues_in = 256
max_length_roles_in = 640

print('cues')
len_enc = [len(e) for e in encodings_cues]
print(f'max length: {max(len_enc)}')
print(f'mean length: {np.mean(len_enc)}')
print(f'number of samples longer than {max_length_cues_in}: {sum(np.array(len_enc) > max_length_cues_in)}')

print('roles')
len_enc = [len(e) for e in encodings_roles]
print(f'max length: {max(len_enc)}')
print(f'mean length: {np.mean(len_enc)}')
print(f'number of samples longer than {max_length_roles_in}: {sum(np.array(len_enc) > max_length_roles_in)}')

cues
323
144.7142249175444
7
roles
648
326.9815691579303
6


In [7]:
# encode all prompt outputs with the Llama 1 tokenizer
encodings_cues_out = []
encodings_roles_out = []
for l in lines_cues:
    enc = tokenizer.encode(json.loads(l)['output'])
    encodings_cues_out.append(enc)
for l in lines_roles:
    enc = tokenizer.encode(json.loads(l)['output'])
    encodings_roles_out.append(enc)

In [8]:
# check the length of the encoded prompt outputs, needed to set the max length of the model in the fine-tuning scripts
max_length_cues_out = 64
max_length_roles_out = 256

print('cues')
len_enc = [len(e) for e in encodings_cues_out]
print(f'max length: {max(len_enc)}')
print(f'mean length: {np.mean(len_enc)}')
print(f'number of samples longer than {max_length_cues_out}: {sum(np.array(len_enc) > max_length_cues_out)}')

print('roles')
len_enc = [len(e) for e in encodings_roles_out]
print(f'max length: {max(len_enc)}')
print(f'mean length: {np.mean(len_enc)}')
print(f'number of samples longer than {max_length_roles_out}: {sum(np.array(len_enc) > max_length_roles_out)}')

cues
50
10.385998510479839
0
roles
259
71.00067636117687
1
